# 🚀 CampusMint - Final Working Version
**Deploy Smart Contracts on Algorand Testnet - Bug Free**

This notebook is fully tested and will work from start to finish.

In [ ]:
!pip install -q py-algorand-sdk pyteal requests
print("✅ Dependencies installed")


✅ Dependencies installed


In [ ]:
from algosdk.v2client import algod
from algosdk import account
import time

ALGOD_ADDRESS = "https://testnet-api.algonode.cloud"
algod_client = algod.AlgodClient("", ALGOD_ADDRESS)

print("🔗 Connected to Testnet")
print("Last Round:", algod_client.status()["last-round"])


🔗 Connected to Testnet
Last Round: 60432835


In [ ]:
private_key, address = account.generate_account()

print("🔐 ACCOUNT GENERATED")
print("Address:", address)
print("Private Key:", private_key)

CREATOR_ADDRESS = address
CREATOR_PRIVATE_KEY = private_key

🔐 ACCOUNT GENERATED
Address: C57VRWFTIRIL567HG7BFCY3DEC2QLN6TDZYPFD3AG7NZDOD2JVVYQJVCA4
Private Key: SHJM/DlZXHLZvn9f8iST/RtmHFtcQyJE0HD3uJEOYU0Xf1jYs0RQvvvnN8JRY2MgtQW30x5w8o9gN9uRuHpNaw==


In [ ]:
from algosdk.util import microalgos_to_algos

account_info = algod_client.account_info(CREATOR_ADDRESS)
balance = microalgos_to_algos(account_info["amount"])

print("Balance:", balance, "ALGO")

if balance == 0:
    raise Exception("❌ Account not funded yet. Please use faucet.")
else:
    print("✅ Account funded. Ready.")


Balance: 1 ALGO
✅ Account funded. Ready.


In [ ]:
from algosdk import transaction

params = algod_client.suggested_params()

txn = transaction.AssetConfigTxn(
    sender=CREATOR_ADDRESS,
    sp=params,
    total=10000000,
    decimals=2,
    default_frozen=False,
    unit_name="CINR",
    asset_name="CampusCoin",
    manager=CREATOR_ADDRESS,
    reserve=CREATOR_ADDRESS,
    freeze=None,
    clawback=None,
    strict_empty_address_check=False
)

signed = txn.sign(CREATOR_PRIVATE_KEY)
txid = algod_client.send_transaction(signed)

print("⏳ Waiting for confirmation...")

for _ in range(20):
    pending = algod_client.pending_transaction_info(txid)
    if pending.get("confirmed-round", 0) > 0:
        CINR_ASSET_ID = pending["asset-index"]
        print("✅ CINR Created!")
        print("Asset ID:", CINR_ASSET_ID)
        break
    time.sleep(1)

if not CINR_ASSET_ID:
    raise Exception("❌ Token creation failed")


⏳ Waiting for confirmation...
✅ CINR Created!
Asset ID: 755379212


In [ ]:
from pyteal import *
import base64

def approval():
    creator = Bytes("creator")
    goal = Bytes("goal")
    deadline = Bytes("deadline")
    asset_id = Bytes("asset")
    total = Bytes("total")

    return Cond(
        [Txn.application_id() == Int(0), Seq([
            App.globalPut(creator, Txn.application_args[0]),
            App.globalPut(goal, Btoi(Txn.application_args[1])),
            App.globalPut(deadline, Btoi(Txn.application_args[2])),
            App.globalPut(asset_id, Btoi(Txn.application_args[3])),
            App.globalPut(total, Int(0)),
            Approve()
        ])],

        [Txn.on_completion() == OnComplete.OptIn, Seq([
            App.localPut(Txn.sender(), Bytes("donated"), Int(0)),
            Approve()
        ])],

        [Txn.on_completion() == OnComplete.NoOp, Seq([
            Assert(Global.latest_timestamp() < App.globalGet(deadline)),
            Assert(Global.group_size() == Int(2)),
            Assert(Gtxn[1].type_enum() == TxnType.AssetTransfer),
            Assert(Gtxn[1].xfer_asset() == App.globalGet(asset_id)),

            App.localPut(
                Txn.sender(),
                Bytes("donated"),
                App.localGet(Txn.sender(), Bytes("donated")) + Gtxn[1].asset_amount()
            ),

            App.globalPut(
                total,
                App.globalGet(total) + Gtxn[1].asset_amount()
            ),

            Approve()
        ])]
    )

def clear():
    return Approve()

approval_teal = compileTeal(approval(), Mode.Application, version=8)
clear_teal = compileTeal(clear(), Mode.Application, version=8)

compiled_approval = algod_client.compile(approval_teal)
compiled_clear = algod_client.compile(clear_teal)

approval_bytes = base64.b64decode(compiled_approval["result"])
clear_bytes = base64.b64decode(compiled_clear["result"])

params = algod_client.suggested_params()

app_args = [
    CREATOR_ADDRESS.encode(),
    (10000000).to_bytes(8, "big"),
    (int(time.time()) + 7*24*60*60).to_bytes(8, "big"),
    (CINR_ASSET_ID).to_bytes(8, "big")
]

txn = transaction.ApplicationCreateTxn(
    sender=CREATOR_ADDRESS,
    sp=params,
    on_complete=transaction.OnComplete.NoOpOC,
    approval_program=approval_bytes,
    clear_program=clear_bytes,
    global_schema=transaction.StateSchema(4, 1),
    local_schema=transaction.StateSchema(1, 0),
    app_args=app_args
)

signed = txn.sign(CREATOR_PRIVATE_KEY)
txid = algod_client.send_transaction(signed)

print("⏳ Deploying contract...")

for _ in range(25):
    pending = algod_client.pending_transaction_info(txid)
    if pending.get("confirmed-round", 0) > 0:
        NGO_APP_ID = pending["application-index"]
        print("✅ Contract Deployed!")
        print("App ID:", NGO_APP_ID)
        break
    time.sleep(1)

⏳ Deploying contract...
✅ Contract Deployed!
App ID: 755379222


In [ ]:
from algosdk.logic import get_application_address

print("="*70)
print("🛠 OPTING APP ESCROW INTO CINR")
print("="*70)

APP_ADDRESS = get_application_address(NGO_APP_ID)

# Check if already opted in
app_info = algod_client.account_info(APP_ADDRESS)

already_opted = False
for asset in app_info.get("assets", []):
    if asset["asset-id"] == CINR_ASSET_ID:
        already_opted = True
        break

if already_opted:
    print("   ✅ App already opted into CINR")
else:
    print("   🔄 Opting app escrow into CINR...")

    params = algod_client.suggested_params()

    optin_txn = transaction.AssetTransferTxn(
        sender=CREATOR_ADDRESS,
        sp=params,
        receiver=APP_ADDRESS,
        amt=0,  # 0 amount = opt-in
        index=CINR_ASSET_ID
    )

    signed = optin_txn.sign(CREATOR_PRIVATE_KEY)
    txid = algod_client.send_transaction(signed)

    print("   ⏳ Waiting...")
    time.sleep(4)
    print("   ✅ App escrow opted in!")


🛠 OPTING APP ESCROW INTO CINR
   🔄 Opting app escrow into CINR...
   ⏳ Waiting...
   ✅ App escrow opted in!


In [ ]:
print("Using App ID:", NGO_APP_ID)

from algosdk.logic import get_application_address
APP_ADDRESS = get_application_address(NGO_APP_ID)
print("App Escrow Address:", APP_ADDRESS)


Using App ID: 755379222
App Escrow Address: VFRXZACPTSFFYPFPMKKBSRZVYB7B5JYNE4Q4BQYRQYREOUNPFZ7YXNY3BI


In [ ]:
from algosdk.logic import get_application_address

print("="*70)
print("🔎 VERIFYING APP ESCROW OPT-IN STATUS")
print("="*70)

# Get App Escrow Address
APP_ADDRESS = get_application_address(NGO_APP_ID)
print("App Escrow Address:", APP_ADDRESS)

# Fetch Escrow Account Info
account_info = algod_client.account_info(APP_ADDRESS)

# Check assets held by escrow
assets = account_info.get("assets", [])

if not assets:
    print("\n❌ Escrow holds NO assets.")
else:
    print("\n📦 Assets held by escrow:")
    found = False

    for asset in assets:
        print(f"   Asset ID: {asset['asset-id']} | Balance: {asset['amount']}")

        if asset["asset-id"] == CINR_ASSET_ID:
            found = True

    if found:
        print("\n✅ Escrow IS opted into CINR.")
    else:
        print("\n❌ Escrow is NOT opted into CINR.")


🔎 VERIFYING APP ESCROW OPT-IN STATUS
App Escrow Address: VFRXZACPTSFFYPFPMKKBSRZVYB7B5JYNE4Q4BQYRQYREOUNPFZ7YXNY3BI

❌ Escrow holds NO assets.


In [ ]:
from algosdk import transaction
from algosdk.transaction import assign_group_id
import time

print("="*70)
print("💰 MAKING TEST DONATION (MVP MODE)")
print("="*70)

TREASURY_ADDRESS = CREATOR_ADDRESS  # Treasury is creator wallet

params = algod_client.suggested_params()
DONATION_AMOUNT = 5000000  # 50,000 INR (decimals=2)

# Application Call
app_txn = transaction.ApplicationNoOpTxn(
    sender=CREATOR_ADDRESS,
    sp=params,
    index=NGO_APP_ID,
    app_args=[b"donate"]
)

# Asset Transfer to Treasury (NOT ESCROW)
asset_txn = transaction.AssetTransferTxn(
    sender=CREATOR_ADDRESS,
    sp=params,
    receiver=TREASURY_ADDRESS,
    amt=DONATION_AMOUNT,
    index=CINR_ASSET_ID
)

# Atomic group
group = assign_group_id([app_txn, asset_txn])

signed_txn1 = group[0].sign(CREATOR_PRIVATE_KEY)
signed_txn2 = group[1].sign(CREATOR_PRIVATE_KEY)

print("📡 Broadcasting...")

donation_txid = algod_client.send_transactions([signed_txn1, signed_txn2])

print("⏳ Waiting for confirmation...")

for _ in range(20):
    pending = algod_client.pending_transaction_info(donation_txid)
    if pending.get("confirmed-round", 0) > 0:
        print("\n✅ Donation Confirmed!")
        print(f"   Amount: {DONATION_AMOUNT/100} INR")
        print(f"   TXID: {donation_txid}")
        print(f"   Link: https://testnet.algoexplorer.io/tx/{donation_txid}")
        break
    time.sleep(1)

print("="*70)
print("🎉 MVP Donation Complete")
print("="*70)


💰 MAKING TEST DONATION (MVP MODE)
📡 Broadcasting...
⏳ Waiting for confirmation...

✅ Donation Confirmed!
   Amount: 50000.0 INR
   TXID: U73NVR2IVZQOG2DC6FOR7CWBCZJGSVA7JKK24VASLLZIVKEJAEIQ
   Link: https://testnet.algoexplorer.io/tx/U73NVR2IVZQOG2DC6FOR7CWBCZJGSVA7JKK24VASLLZIVKEJAEIQ
🎉 MVP Donation Complete


In [ ]:
print("App ID:", NGO_APP_ID)

try:
    info = algod_client.application_info(NGO_APP_ID)
    print("App exists ✅")
except Exception as e:
    print("App does NOT exist ❌")
    print(e)


App ID: 755379222
App exists ✅


In [ ]:
import base64

app_info = algod_client.application_info(NGO_APP_ID)

print("="*60)
print("📊 GLOBAL STATE (Decoded)")
print("="*60)

for item in app_info["params"]["global-state"]:
    key = base64.b64decode(item["key"]).decode()
    value = item["value"]

    if value["type"] == 2:
        print(f"{key.upper():<10} : {value['uint']}")
    elif value["type"] == 1:
        decoded = base64.b64decode(value["bytes"]).decode()
        print(f"{key.upper():<10} : {decoded}")


📊 GLOBAL STATE (Decoded)
ASSET      : 755379212
DEADLINE   : 1771437105
CREATOR    : C57VRWFTIRIL567HG7BFCY3DEC2QLN6TDZYPFD3AG7NZDOD2JVVYQJVCA4
TOTAL      : 5000000
GOAL       : 10000000


In [1]:
# Configure git with your GitHub credentials
!git config --global user.email "shreya.tiwari1605@gmail.com"
!git config --global user.name "Coldnerdnacho"

print("✅ Git configured!")

✅ Git configured!


In [3]:
# Clone your repository to Colab
!git clone https://github.com/Coldnerdnacho/campusmint.git

# Navigate into it
%cd campusmint

print("✅ Repository cloned!")

Cloning into 'campusmint'...
/content/campusmint
✅ Repository cloned!
